<a href="https://colab.research.google.com/github/John-zf/SMAML_for_Tool_Wear_Monitoring-Prediction/blob/master/SMAML_for_Experiments_c6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import torch

print(torch.cuda.get_device_name(0))
drive.mount('/content/drive/', force_remount=True)

Tesla T4
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Load data and transform them into 2d images
import pandas as pd
import numpy as np
import os

def get_data(data_folder):
  # Read the sensor data
  data = []
  for file in os.listdir(data_folder):
    file_path = data_folder + file
    ins = pd.read_csv(file_path, header=None)
    ins = np.array(ins).T[:, :224*224]
    # Centralization
    ins = np.array([x-np.mean(x) for x in ins])
    data.append(ins)
  data = np.array(data)
  print('{} reading is done.'.format(data_folder))
  return data

data_folder_c1 = '/content/drive/My Drive/mhm_data/c1/c1/'
data_folder_c4 = '/content/drive/My Drive/mhm_data/c4/c4/'
data_folder_c6 = '/content/drive/My Drive/mhm_data/c6/c6/'

data_c1 = get_data(data_folder_c1)
data_c4 = get_data(data_folder_c4)
data_c6 = get_data(data_folder_c6) 

/content/drive/My Drive/mhm_data/c1/c1/ reading is done.
/content/drive/My Drive/mhm_data/c4/c4/ reading is done.
/content/drive/My Drive/mhm_data/c6/c6/ reading is done.


In [ ]:
# Load target
def get_target(target_file, step=10):
    # Read target, 
    target = pd.read_csv(target_file)
    target = np.array(target[['flute_1', 'flute_2', 'flute_3']])
    target = [x[0] for x in target]
    print(target)
    if step == 1:
      return target
    else:
      targets_list = []
      for i, value in enumerate(target):
        targets = []
        for j in range(i, (i+step)):
          if j > len(target)-1:
            targets.append(target[-1])
          else:
            targets.append(target[j])
        targets_list.append(targets)
      return np.array(targets_list)

target_file_c1 = '/content/drive/My Drive/mhm_data/c1/c1_wear.csv'
target_file_c4 = '/content/drive/My Drive/mhm_data/c4/c4_wear.csv'
target_file_c6 = '/content/drive/My Drive/mhm_data/c6/c6_wear.csv'

target_c1 = get_target(target_file_c1)
target_c4 = get_target(target_file_c4)
target_c6 = get_target(target_file_c6)

[32.31711361, 37.914879, 43.08790971, 47.8590723, 52.25032922, 56.28276562, 59.97661561, 63.351287899999996, 66.4253909, 69.21675719, 71.74246752, 74.01887416, 76.06162381, 77.88567991, 79.50534445, 80.93427925, 82.18552678, 83.27153045, 84.20415441, 84.99470288, 85.65393900000001, 86.19210327, 86.61893145, 86.94367205, 87.17510345, 87.32155046, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87.39090058, 87

In [ ]:
# split data

######## Training and Test Data ########
train_fea = [data_c1, data_c4]
train_tar = [target_c1, target_c4]
test_fea = data_c6
test_tar = target_c6
#############################################

train_data = np.vstack(train_fea)
train_label = np.vstack(train_tar)

In [ ]:
# Traditional feature extraction
import numpy as np
import scipy.stats as sts
from pywt import WaveletPacket

def rms_fea(a):
	return np.sqrt(np.mean(np.square(a)))

def var_fea(a):
	return np.var(a)

def max_fea(a):
	return np.max(a)

def pp_fea(a):
	return np.max(a)-np.min(a)

def skew_fea(a):
	return sts.skew(a)

def kurt_fea(a):
	return sts.kurtosis(a)

def wave_fea(a):
	wp = WaveletPacket(a,'db1', maxlevel=8)
	nodes = wp.get_level(8, "freq")
	return np.linalg.norm(np.array([n.data for n in nodes]), 2)

def spectral_kurt(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1: int(N/2)] * 2.00/N
	return sts.kurtosis(mag)

def spectral_skw(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1:int(N/2)]*2.00/N
	return sts.skew(mag)

def spectral_pow(a):
	N = len(a)
	mag = np.abs(np.fft.fft(a))
	mag	= mag[1:int(N/2)]*2.00/N
	return np.mean(np.power(mag, 3))
 
def extract_fea(data):
	data_fea = []
	for i in range(7):
		data_fea.append(rms_fea(data[i]))
		data_fea.append(var_fea(data[i]))
		data_fea.append(max_fea(data[i]))
		data_fea.append(pp_fea(data[i]))
		data_fea.append(skew_fea(data[i]))
		data_fea.append(kurt_fea(data[i]))
		data_fea.append(wave_fea(data[i]))
		data_fea.append(spectral_kurt(data[i]))
		data_fea.append(spectral_skw(data[i]))
		data_fea.append(spectral_pow(data[i]))
	return np.array(data_fea)
 
trian_data_ml = np.array([extract_fea(x) for x in train_data])
test_data_ml = np.array([extract_fea(x) for x in test_fea])

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# setp = 0, 1, 5, 9
setp = 9
train_label_ml = train_label[:, setp]
test_label_ml = test_tar[:, setp]

scaler = StandardScaler()
scaler.fit(trian_data_ml)
train_data_sc = scaler.transform(trian_data_ml)
test_data_sc = scaler.transform(test_data_ml)

# SVR
regr = SVR(kernel='rbf')
regr.fit(train_data_sc, train_label_ml)
preds = regr.predict(test_data_sc)
print('SVR -> {:.2f}  {:.2f}'.format(mean_squared_error(test_label_ml, preds), mean_absolute_error(test_label_ml, preds)))

# XGBoost
seed = 2020
for i in range(10):
  regr = XGBRegressor(
      learning_rate=0.1,
      n_estimators=200,
      max_depth=1,
      subsample=0.8,
      colsample_bytree=0.8,
      objective='reg:squarederror',
      random_state=seed+i)
  regr.fit(train_data_sc, train_label_ml)
  preds = regr.predict(test_data_sc)
  print('{:.2f} {:.2f}'.format(mean_squared_error(test_label_ml, preds), mean_absolute_error(test_label_ml, preds)))

SVR -> 797.14  21.48
408.12 16.16
424.12 16.85
397.47 16.08
355.93 15.22
450.70 17.34
324.53 14.21
371.73 15.48
356.12 15.27
473.06 17.64
334.34 14.63


In [ ]:
# For Pytorch
import random
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split


def seed_torch(seed=2020):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
seed_torch()

train_data_dl = torch.from_numpy(train_data).float()
train_label_dl = torch.from_numpy(train_label).float()
deal_dataset = TensorDataset(train_data_dl, train_label_dl)

# Split training and validation data
ratio = 0.9
num_train = int(ratio * len(deal_dataset))
num_val = len(deal_dataset) - num_train
train_data_dl, val_data_dl = random_split(deal_dataset, [num_train, num_val])
print('train:', len(train_data_dl), 'validation:', len(val_data_dl))
train_loader = DataLoader(dataset=train_data_dl, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data_dl, batch_size=16, shuffle=True)

# Test data
test_data_dl = torch.from_numpy(test_fea).float()
test_label_dl = torch.from_numpy(test_tar).float()

deal_test = TensorDataset(test_data_dl, test_label_dl)
test_loader = DataLoader(dataset=deal_test, batch_size=16, shuffle=False)
print('test:', len(deal_test))

train: 567 validation: 63
test: 315


In [ ]:
# Define Mono Loss Function
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class MonoLossFunc(nn.Module):
  def __init__(self, lamb=1):
    super(MonoLossFunc, self).__init__()
    self.lamb = lamb

  def forward(self, prediction, truth):
    # calculate mse
    mse_func = torch.nn.MSELoss(reduction='sum')
    loss1 = mse_func(prediction, truth)

    # calculate monotonicity
    data1 = prediction[:, : -1]
    data2 = prediction[:, 1 :]
    loss2 = torch.sum(torch.clamp(data1 - data2, min=0))
    return loss1 + self.lamb * loss2


def evaluate(model, hidden, criteria, val_loader, test_loader):
  model.eval()
  val_loss = 0.0
  # hidden = model.hidden
  hidden = model.initHidden()
  for i, (inputs, labels) in enumerate(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, _ = model(inputs, hidden)
    loss = criteria(output, labels)
    val_loss += loss.item()
  val_loss = val_loss/63

  mse_0 = 0.0
  mae_0 = 0.0
  mse_1 = 0.0
  mae_1 = 0.0
  mse_5 = 0.0
  mae_5 = 0.0
  mse_9 = 0.0
  mae_9 = 0.0

  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, weights = model(inputs, hidden)
    mse_0 += torch.sum(torch.pow((output[:, 0] - labels[:, 0]), 2)).item()
    mae_0 += torch.sum(torch.abs(output[:, 0] - labels[:, 0])).item()

    mse_1 += torch.sum(torch.pow((output[:, 1] - labels[:, 1]), 2)).item()
    mae_1 += torch.sum(torch.abs(output[:, 1] - labels[:, 1])).item()

    mse_5 += torch.sum(torch.pow((output[:, 5] - labels[:, 5]), 2)).item()
    mae_5 += torch.sum(torch.abs(output[:, 5] - labels[:, 5])).item()

    mse_9 += torch.sum(torch.pow((output[:, 9] - labels[:, 9]), 2)).item()
    mae_9 += torch.sum(torch.abs(output[:, 9] - labels[:, 9])).item()

  return val_loss, weights, mse_0/315, mae_0/315, mse_1/315, mae_1/315,\
      mse_5/315, mae_5/315, mse_9/315, mae_9/315


def train(model, lamb=0, max_iter=200):
  learning_rate = 0.001
  model.to(device)
  # hidden = model.hidden

  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.9)
  criteria = MonoLossFunc(lamb=lamb)

  # training the model
  for epoch in range(max_iter):
    model.train()

    hidden = model.initHidden()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      output, _ = model(inputs, hidden)
      loss = criteria(output, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
    train_loss = train_loss/567
    val_loss, weights, mse_0, mae_0, mse_1, mae_1, mse_5, mae_5, \
    mse_9, mae_9 = evaluate(model, hidden, criteria, val_loader, test_loader)
    print('EPOCH={} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}\
    {:.6f} {:.6f} {:.6f}'.format(epoch+1, train_loss, val_loss, \
    mse_0, mae_0, mse_1, mae_1, mse_5, mae_5, mse_9, mae_9))

In [ ]:
# CNN
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
          nn.Linear(256*10, 10),
        )

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    output = self.encoder(input)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 28473.481189 80908.357143 3009.630413 29.613895 1596.897835 23.713267 2663.746216    30.599946 1852.529748 26.036646
EPOCH=2 3814.474525 931.896624 877.889170 25.989238 1032.203163 28.980847 881.160769    25.204241 909.407884 26.599362
EPOCH=3 4292.999154 9266.145585 1005.703498 28.637432 1180.105407 31.271735 1148.796527    31.415522 1566.831602 37.621548
EPOCH=4 3239.628870 1171.444475 1179.564708 22.368032 984.123409 26.396745 965.082763    23.165338 737.153145 21.519520
EPOCH=5 3008.489899 5719.005828 136.281782 10.263408 621.613446 23.251533 449.342333    19.460672 645.887218 23.534631
EPOCH=6 2037.089255 2061.388641 1235.705740 27.187300 1063.359964 28.576047 887.161136    25.450017 814.114360 26.233729
EPOCH=7 1878.440395 1126.608026 372.265968 16.624512 587.609329 21.929010 385.046618    17.837586 407.061540 18.307478
EPOCH=8 2384.009493 2890.211000 930.129803 28.459030 582.768031 21.775596 623.640577    19.788380 656.768033 18.740650
EPOCH=9 1454.087902 1194.848431 484

In [ ]:
# G-CNN
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
          nn.Linear(256*10, 10),
        )

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    output = self.encoder(input)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 11128.359781 60934.421627 9234.052803 95.654972 9515.758929 97.105412 9660.977381    97.891239 9353.745263 96.307110
EPOCH=2 1612.496291 22652.490327 3439.587841 57.849839 3324.983371 56.826732 3263.543657    56.336941 2719.317479 50.884100
EPOCH=3 1635.237641 5363.693080 540.903612 20.248905 547.955847 20.105187 259.479539    13.503761 1048.763190 27.688035
EPOCH=4 1767.045469 2358.467262 497.584229 17.279251 643.521418 19.657541 1291.417948    28.144933 1821.922036 37.224715
EPOCH=5 727.026618 421.302386 752.740402 22.174474 1030.107860 25.757368 2068.399713    35.663957 2770.745138 42.028925
EPOCH=6 683.318842 10673.719246 4468.239723 58.944841 6232.938876 70.098363 8205.136908    80.171183 6168.580246 68.527787
EPOCH=7 498.419922 5559.156250 300.937270 14.068035 452.143082 17.552712 881.558679    24.090999 645.879766 20.428011
EPOCH=8 493.965391 7423.169271 417.427650 16.975123 473.570148 18.251805 1355.548852    29.648246 1445.182196 30.236537
EPOCH=9 676.540703 866.983135

In [ ]:
# GRU
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.avg_pool = nn.AvgPool1d(kernel_size=7)
    self.batch_norm = nn.BatchNorm1d(224*224)
    self.gru = nn.GRU(1568, 256, batch_first=True, bidirectional=False)
    self.out = nn.Linear(256, 10)

  def forward(self, input, hidden):
    input = input.permute(0, 2, 1)
    input = self.avg_pool(input)
    input = input.squeeze(2)
    input = self.batch_norm(input)

    input = input.view(-1, 32, 1568)
    gru_out, _ = self.gru(input)
    output = gru_out[:, -1, :]
    output = self.out(output)
    return output, 1
  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 112460.118827 117253.194444 13159.766840 111.372857 13429.640935 112.583102 13518.346280    112.989230 13978.379067 115.014455
EPOCH=2 93648.864638 99057.871032 11185.216518 102.119191 11430.863318 103.321063 11511.954812    103.727714 11932.294816 105.742397
EPOCH=3 78149.188768 83689.216270 9505.604687 93.534487 9727.824343 94.721619 9800.811936    95.122451 10182.623103 97.117272
EPOCH=4 65169.672840 70615.102183 8065.864503 85.492439 8265.376687 86.658727 8329.952784    87.048376 8673.638467 89.009986
EPOCH=5 54203.207727 59762.772817 6860.578807 78.125994 7038.239896 79.262833 7094.818056    79.638459 7402.184673 81.555645
EPOCH=6 45190.927138 50416.415675 5811.789267 71.097603 5968.471363 72.199835 6016.812461    72.555266 6288.932363 74.418136
EPOCH=7 37675.669174 42733.437500 4939.144071 64.670158 5076.276927 65.731479 5117.372566    66.066878 5356.300794 67.863340
EPOCH=8 31561.600777 36418.626984 4213.073226 58.795616 4331.735928 59.800369 4365.334008    60.106506 457

In [ ]:
# Bi-GRU
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.avg_pool = nn.AvgPool1d(kernel_size=7)
    self.batch_norm = nn.BatchNorm1d(224*224)
    self.gru = nn.GRU(1568, 256, batch_first=True, bidirectional=True)
    self.out = nn.Linear(256*2, 10)

  def forward(self, input, hidden):
    input = input.permute(0, 2, 1)
    input = self.avg_pool(input)
    input = input.squeeze(2)
    input = self.batch_norm(input)

    input = input.view(-1, 32, 1568)
    gru_out, _ = self.gru(input)
    output = gru_out[:, -1, :]
    output = self.out(output)
    return output, 1
  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 108179.340057 106540.607143 12001.540985 106.249910 12097.806448 106.719017 12479.962450    108.498937 12611.469469 109.122936
EPOCH=2 80100.342152 79444.377976 9046.709195 91.344006 9122.163430 91.766891 9452.792423    93.561993 9559.907688 94.149487
EPOCH=3 58442.411486 58080.315476 6690.153144 77.232015 6750.500273 77.625262 7016.632729    79.339370 7101.731331 79.892673
EPOCH=4 41945.643519 42902.617063 4958.901812 64.773390 5003.933112 65.124344 5217.919885    66.773459 5279.276959 67.244625
EPOCH=5 29908.229773 31551.888393 3627.027704 53.345188 3659.222052 53.584548 3813.574252    54.849569 3858.960225 55.220348
EPOCH=6 22109.630401 24826.972718 2775.248688 45.731648 2796.939069 45.888418 2906.405502    46.762207 2934.433111 46.931887
EPOCH=7 17493.941523 20694.396329 2237.722613 40.670881 2250.302328 40.743124 2322.499717    41.262442 2343.182726 41.276994
EPOCH=8 14649.280068 17050.838046 1915.943413 37.467783 1926.272471 37.545963 1980.052310    37.978226 1996.346625 

In [ ]:
# M1
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),

          nn.Flatten(),
          nn.Linear(7*7*7*72, 256*10),
        )
    self.dropout = nn.Dropout(0.5)

    self.gru = nn.GRU(self.hidden_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.out = nn.Linear(self.hidden_size*2, 1)

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    encoder_outputs = self.encoder(input)
    encoder_outputs = self.dropout(encoder_outputs)
    print
    encoder_outputs = encoder_outputs.view(-1, 10, 256)
    
    hidden, _ = self.gru(encoder_outputs)
    output = self.out(hidden)
    output = output.squeeze(2)
    return output, 1

  def initHidden(self):
    return torch.zeros(16, self.output_size, self.hidden_size*2, device=device)

model = Model()
train(model)

EPOCH=1 94960.113371 84810.829365 9687.070362 94.536678 9720.227939 94.688302 9988.243583    96.108143 10280.100099 97.641462
EPOCH=2 58983.059689 56588.004960 6516.627561 75.893163 6562.695018 76.203724 6781.436260    77.645587 6998.183098 79.039655
EPOCH=3 38215.224826 38253.705357 4419.393268 60.666326 4454.942208 60.904489 4629.198422    62.327403 4809.377534 63.804845
EPOCH=4 25596.678654 26950.851190 3062.674360 48.489701 3097.427035 48.804980 3235.961584    50.025065 3396.225827 51.424553
EPOCH=5 18181.162202 20136.315476 2264.116367 40.920198 2256.184179 40.654999 2378.995447    41.825145 2497.014394 42.938752
EPOCH=6 13787.966104 16120.039187 1673.527089 34.353314 1705.225598 34.849651 1792.304548    35.578073 1885.418570 36.488647
EPOCH=7 11007.108156 13362.449157 1314.414336 30.113503 1327.031211 29.879433 1407.633020    31.021997 1630.505352 33.007116
EPOCH=8 9204.114876 11361.594494 1042.806391 26.039340 1069.689087 26.070122 1307.941534    28.810008 1209.190949 27.996550


KeyboardInterrupt: ignored

In [ ]:
# SMAML
class Model(nn.Module):
  def __init__(self, hidden_size=256, output_size=10):
    super(Model, self).__init__()
    # Encoder
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.encoder = nn.Sequential(
          # depthwise convolution
          nn.Conv2d(7, 7*9, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 112*112

          nn.Conv2d(7*9, 7*18, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 56*56

          # group convolution
          nn.Conv2d(7*18, 7*36, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 28*28

          # group convolution
          nn.Conv2d(7*36, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 14*14

          # depthwise convolution
          nn.Conv2d(7*72, 7*72, 3, groups=7),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d((2, 2), padding=(1, 1)), # 7*7
          nn.BatchNorm2d(7*72),
        )
    self.linear1 = nn.Linear(7*7*72, self.hidden_size)
    self.batch_norm = nn.BatchNorm1d(self.hidden_size)

    # Decoder
    self.linear2 = nn.Linear(224*224, self.hidden_size*2)
    self.dropout = nn.Dropout(0.8)

    self.gru = nn.GRU(self.hidden_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.out = nn.Linear(self.hidden_size*2, 1)

  def forward(self, input, hidden):
    input = input.view(-1, 7, 224, 224)
    # Encoder
    encoder_outputs = self.encoder(input)
    encoder_outputs = encoder_outputs.view(-1, 7, 7*7*72)
    encoder_outputs = self.dropout(self.linear1(encoder_outputs))

    # Decoder with attention
    input = input.view(-1, 7, 224*224)
    embed = torch.tanh(self.linear2(input))
    embed = self.dropout(embed)

    hidden = hidden[: len(embed)]
    attn_weights = F.softmax(torch.bmm(hidden, embed.permute(0, 2, 1)), dim=2)
    attn_applied = F.relu(torch.bmm(attn_weights, encoder_outputs))
    attn_applied = self.batch_norm(attn_applied.permute(0, 2, 1))
    attn_applied = attn_applied.permute(0, 2, 1)

    hidden, _ = self.gru(attn_applied)
    output = self.out(hidden)
    output = output.squeeze(2)
    return output, attn_weights

  def initHidden(self):
    return torch.randn(16, self.output_size, self.hidden_size*2, device=device)


model = Model()
train(model, lamb=5, max_iter=131)

EPOCH=1 105377.378968 92586.131944 11786.572532 105.254748 10798.487785 100.687477 10329.073289    98.236876 12461.344407 108.512034
EPOCH=2 62686.427938 58568.854167 8172.694575 86.744173 6730.227840 78.006238 6564.599194    76.473298 9057.342448 91.601543
EPOCH=3 38731.645172 38595.117560 5617.163294 70.959270 4533.583811 62.689776 4307.274960    59.840437 6124.883380 73.863558
EPOCH=4 24683.182457 25982.669643 3807.699679 56.205609 2868.197967 47.152132 2788.337340    45.818816 4177.375307 58.703040
EPOCH=5 17001.712591 18661.704117 2649.875569 45.967736 2031.336079 38.877777 1984.884757    37.775015 2820.174619 47.580225
EPOCH=6 13449.450741 15702.983879 2435.620055 44.381934 1721.085193 35.562455 1583.360288    33.285844 2519.461204 44.385225
EPOCH=7 11999.210627 18554.610615 2743.543166 44.153347 1686.444054 34.069910 1309.437419    29.324194 3138.094032 46.524432
EPOCH=8 11203.939746 11463.379712 2458.249721 40.791671 1437.135375 30.197116 1082.144598    26.408231 2414.118621 40

In [ ]:
def pred(model, val_loader, test_loader):
  model.eval()
  hidden = model.initHidden()
  pred_label = []
  mse = 0.0
  weights = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    output, weight = model(inputs, hidden)
    weights.append(weight.to('cpu').detach().numpy())
    pred_label.append(output.to('cpu').detach().numpy())
    step = 0
    labels = labels[:, step]
    output = output[:, step]
    mse_sum = torch.sum(torch.pow((output - labels), 2))
    mse += mse_sum.item()
  mse = mse/315
  return np.vstack(pred_label), mse, np.array(weights)

preds, mse, weights = pred(model, val_loader, test_loader)
print(mse)
for k, i in enumerate(preds):
  print(','.join([str(k) for k in list(i)]))

652.6665201822917
54.161953,50.797894,51.371235,48.556538,51.541126,51.49661,51.07189,53.595276,62.04542,64.447495
69.213905,69.65639,71.258675,78.95625,79.927376,81.78091,81.98569,82.29977,82.56979,78.99101
80.496826,81.98875,90.939445,91.256256,91.05977,93.27094,93.30029,94.65586,98.00924,99.31447
71.885796,71.98522,72.99913,80.096695,79.68683,84.3447,84.627045,86.01364,85.149445,75.767006
85.9191,86.69067,86.62791,86.3087,85.864655,86.30152,87.86499,88.75788,89.10874,91.45346
64.21728,66.64737,74.95161,76.86258,78.08464,79.60889,78.530235,80.42536,85.943054,89.65602
73.367134,69.332245,79.9817,80.26177,83.57891,85.203995,81.2507,74.850815,74.612755,81.13349
90.19632,83.660706,84.15597,80.51285,83.9252,83.9362,84.012085,79.63362,72.87863,74.50277
80.06638,88.14057,88.10521,89.68439,86.91007,86.899826,87.1548,88.31587,90.827385,94.83427
70.99098,84.78685,87.581665,88.12326,88.749054,86.94429,86.66147,91.31972,93.765114,96.40128
93.01475,89.19153,86.02478,87.46665,85.314674,86.70267,86

In [ ]:
for i in weights[10][10]:
  print(','.join([str(round(x, 4)) for x in i]))

0.0,0.5817,0.0,0.0014,0.0,0.0,0.4169
0.0,0.9999,1e-04,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0009,0.0,0.0,0.9991,0.0
0.032,0.0,0.0004,0.0,0.9676,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1e-04,0.9999
0.0,0.0,0.003,0.0,0.0,0.0,0.997
0.9996,0.0,0.0,0.0,0.0004,0.0,0.0
0.0,0.0013,0.9985,0.0,0.0,0.0002,0.0
0.0,0.0,0.0003,0.0,0.0,0.0,0.9996
